<!-- dom:TITLE: Demo - Some fast transforms -->
# Demo - Some fast transforms
<!-- dom:AUTHOR: Mikael Mortensen Email:mikaem@math.uio.no at Department of Mathematics, University of Oslo. -->
<!-- Author: -->  
**Mikael Mortensen** (email: `mikaem@math.uio.no`), Department of Mathematics, University of Oslo.

Date: **Mar 22, 2021**

Copyright 2021, Mikael Mortensen. Released under CC Attribution 4.0 license

**Summary.** This demo shows how to compute fast forward transforms for three
different Dirichlet bases.












## The fast transforms

We will consider the fast forward transforms for the three
Chebyshev Dirichlet basis functions

<!-- Equation labels as ordinary links -->
<div id="eq:shen"></div>

$$
\label{eq:shen} \tag{1}
\phi_k = T_k-T_{k+2}, \quad k=0,1, \ldots, N-3,
$$

<!-- Equation labels as ordinary links -->
<div id="eq:heinrichs"></div>

$$
\label{eq:heinrichs} \tag{2}
\varphi_k = (1-x^2)T_k, \quad k=0,1, \ldots, N-3,
$$

<!-- Equation labels as ordinary links -->
<div id="eq:dirichletU"></div>

$$
\label{eq:dirichletU} \tag{3}
\psi_k = U_k-\frac{k+1}{k+3}U_{k+2}, \quad k=0,1, \ldots, N-3,
$$

That is, we test fast methods for projection of a function
$u(x)$ into spaces $V_{\phi} = \text{span}\{\phi_k\}_{k=0}^{N-3}$,
$V_{\varphi} = \text{span}\{\varphi_k\}_{k=0}^{N-3}$ and
$V_{\psi} = \text{span}\{\psi_k\}_{k=0}^{N-3}$.
For each space the projection is:
find $u_N \in V_{\alpha}$ such that

<!-- Equation labels as ordinary links -->
<div id="eq:projection"></div>

$$
\label{eq:projection} \tag{4}
(u_N-u, v)_{\omega} = 0, \quad \forall \, v \in V_{\alpha},
$$

where $\alpha \in (\phi, \varphi, \psi)$. The solutions we are after are

$$
u_N(x) = \sum_{k=0}^{N-3} \hat{u}_k^{\phi} \phi_k = \sum_{k=0}^{N-3} \hat{u}_k^{\varphi} \varphi_k = \sum_{k=0}^{N-3} \hat{u}_k^{\psi} \psi_k,
$$

where $\hat{u}_k^{\alpha}, \, \alpha \in (\phi, \varphi, \psi)$ are the unknown degrees of freedom. Generically, we write

$$
u(x) = \sum_{k=0}^{N-3} \hat{u}_k \xi_k(x),
$$

where $\xi_k$ can be any of the three basis functions $(\phi_k, \varphi_k, \psi_k)$.

A direct method to solve Eq. ([4](#eq:projection)) is to insert for $u_N = \sum_{k=0}^{N-3}\hat{u}_k \xi_k$ and $v = \xi_j$ to obtain

$$
(\xi_j, \xi_k)_{\omega} \hat{u}_j = (u, \xi_k)_{\omega},
$$

which on algebraic form is

$$
\begin{align*}
B \hat{\mathbf{u}} &= \tilde{\mathbf{u}}, \\ 
\hat{\mathbf{u}} &= B^{-1} \tilde{\mathbf{u}},
\end{align*}
$$

where the mass matrix $B \in \mathbb{R}^{N-2 \times N-2} = \{(\xi_j, \xi_k)_{\omega}\}_{k,j=0}^{N-3}$, $\tilde{\mathbf{u}} = \{(u, \xi_k)_{\omega}\}_{k=0}^{N-3}$ and $\mathbf{u} = \{\hat{u}_k\}_{k=0}^{N-3}$. A fast method is found if $B$ is diagonal, so that the inversion $B^{-1}$ is trivial. In this case the projection becomes orthogonal. It can be shown that the three bases are orthogonal using weights $\omega = (1-x^2)^{\sigma}$, where $\sigma = -3/2, -5/2$ and $-1/2$, for ([1](#eq:shen)), ([2](#eq:heinrichs)) and ([3](#eq:dirichletU)), respectively.

In the following all methods will make use of Gauss-Chebyshev quadrature with Gauss-Chebyshev collocation points $\boldsymbol{x}=\{x_i\}_{i=0}^{N-1}$, where $x_i = \cos(\theta_i)$, $\theta_i=\pi (2i+1)/(2N)$ and $\mathbf{\theta}=\{\theta_i\}_{i=0}^{N-1}$. The quadrature weights are then constant $w_i = \pi/N$ for $i=0,1, \ldots, N-1$.

The fast orthogonal projection for the Dirichlet basis ([1](#eq:shen)) simplifies to

<!-- Equation labels as ordinary links -->
<div id="eq:sol0"></div>

$$
\label{eq:sol0} \tag{5}
\hat{u}_k^{\phi} = \frac{1}{2N} \text{dst}^{II}(\mathbf{u}/\sin \mathbf{\theta})_k, \quad k=0,1, \ldots, N-3.
$$

For basis ([2](#eq:heinrichs)) it is

$$
\hat{u}_k^{\varphi} = \frac{1}{c_k N} \text{dct}^{II} (\mathbf{u}/\sin^2 \mathbf{\theta})_k, \quad k=0, 1, \ldots, N-3,
$$

and for the basis ([3](#eq:dirichletU)) it is

$$
\hat{u}_k^{\psi} = \frac{k+3}{2(k+1)}\hat{u}_k^{\phi} - \frac{1}{2}\hat{u}_{k+2}^{\phi}, \quad k=0, 1, \ldots, N-3.
$$

## Implementation

To validate these methods we compute the projection first regularly
using the Shenfun function [project](https://github.com/spectralDNS/shenfun/blob/master/shenfun/forms/project.py),
which is using $\sigma=-1/2$, and then using the fast methods above.

Start by importing necessary modules from Shenfun and mpi4py-fft

In [1]:
from shenfun import *
from mpi4py_fft import fftw

Create function spaces for all three Dirichlet bases. Decide on a number of quadrature points (not important), and just use default quadrature scheme, which is the Gauss-Chebyshev points.

In [2]:
N = 20
D0 = FunctionSpace(N, 'C', basis='ShenDirichlet')
D1 = FunctionSpace(N, 'C', basis='Heinrichs')
D2 = FunctionSpace(N, 'C', basis='DirichletU')

Create a random vector that we will use for testing.

In [3]:
f = Function(D0, buffer=np.random.random(N))
f[-2:] = 0
f[:] = f.backward().forward()
fb = f.backward().copy()

Do the regular projection. Now `f0`, `f1` and `f2` will be the three solution vectors $\mathbf{\hat{u}}^{\phi}$, $\mathbf{\hat{u}}^{\varphi}$ and $\mathbf{\hat{u}}^{\psi}$.

In [4]:
f0 = project(fb, D0)
f1 = project(fb, D1)
f2 = project(fb, D2)

Now compute the fast transforms and assert that they are equal to `f0`, `f1` and `f2`

In [5]:
theta = np.pi*(2*np.arange(N)+1)/(2*N)
dst = fftw.dstn(fb, type=2)
d0 = dst(fb/np.sin(theta))/(2*N)
assert np.linalg.norm(d0-f0) < 1e-8
dct = fftw.dctn(fb, type=2)
ck = np.ones(N); ck[0] = 2
d1 = dct(fb/np.sin(theta)**2)/(ck*N)
assert np.linalg.norm(d1-f1) < 1e-8, np.linalg.norm(d1-f1)
ut = d0
k = np.arange(N)
d2 = Function(D2)
d2[:-2] = (k[:-2]+3)/2/(k[:-2]+1)*ut[:-2]
d2[:-2] = d2[:-2] - 0.5*ut[2:]
assert np.linalg.norm(d2-f2) < 1e-8

That's it! If you make it to here with no errors, then the three tests pass, and the fast transforms are equal to the slow ones, at least within given precision.

Let's try some timings

In [6]:
%timeit project(fb, D0)

39.9 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%timeit dst(fb/np.sin(theta))/(2*N)

4.3 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


We can precompute the sine term, because it does not change

In [8]:
dd = np.sin(theta)*2*N
%timeit dst(fb/dd)

2.54 µs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The other two transforms are about equally fast

In [9]:
%timeit dct(fb/np.sin(theta)**2)/(ck*N)

5.5 µs ± 33.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<!-- ======= Bibliography ======= -->